In [4]:
#### cross_entropy, one_hot, reshape ####

import tensorflow as tf
import numpy as np
tf.set_random_seed(777)

# 16개의 데이터를 통해 동물의 종(0 ~ 6)을 나타내는 데이터
xy = np.loadtxt('data/data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

print(x_data.shape, y_data.shape)
# (101, 16) (101, 1)

nb_classes = 7  # 0 ~ 6

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])  # 0 ~ 6, shape = (?, 1)

# Y를 one hot 형태로 변환. rank가 1 늘어남.
Y_one_hot = tf.one_hot(Y, nb_classes)  # shape = (?, 1, 7)
print("one_hot :", Y_one_hot)

# rank를 다시 줄여줌. 우리가 원하는 형태.
# [[1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0], ...]
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])  # shape = (?, 7)
print("reshape :", Y_one_hot)

W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)  # softmax

# Cross entropy cost
# 기존 코드 
# cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
# 를 함수로 만들어 놓은 것.
# hypothesis가 아닌 logits을 인자로 받음!!
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                 labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# 예측값
prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2000):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 100 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={
                                 X: x_data, Y: y_data})
            print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(
                step, loss, acc))

    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X: x_data})
    # y_data: (N,1) = flatten => (N, ) matches pred.shape
    for p, y in zip(pred, y_data.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

(101, 16) (101, 1)
one_hot : Tensor("one_hot_2:0", shape=(?, 1, 7), dtype=float32)
reshape : Tensor("Reshape_2:0", shape=(?, 7), dtype=float32)
Step:     0	Loss: 4.024	Acc: 18.81%
Step:   100	Loss: 0.542	Acc: 87.13%
Step:   200	Loss: 0.389	Acc: 88.12%
Step:   300	Loss: 0.305	Acc: 90.10%
Step:   400	Loss: 0.250	Acc: 92.08%
Step:   500	Loss: 0.210	Acc: 94.06%
Step:   600	Loss: 0.181	Acc: 97.03%
Step:   700	Loss: 0.159	Acc: 97.03%
Step:   800	Loss: 0.141	Acc: 98.02%
Step:   900	Loss: 0.126	Acc: 98.02%
Step:  1000	Loss: 0.114	Acc: 99.01%
Step:  1100	Loss: 0.104	Acc: 99.01%
Step:  1200	Loss: 0.096	Acc: 100.00%
Step:  1300	Loss: 0.089	Acc: 100.00%
Step:  1400	Loss: 0.082	Acc: 100.00%
Step:  1500	Loss: 0.077	Acc: 100.00%
Step:  1600	Loss: 0.072	Acc: 100.00%
Step:  1700	Loss: 0.068	Acc: 100.00%
Step:  1800	Loss: 0.064	Acc: 100.00%
Step:  1900	Loss: 0.061	Acc: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[Tru